<p align="center">
  <h1 align="center">🌊 GradTracer v0.6 — AutoFix & Ranking Validation</h1>
  <p align="center">
    <strong>End-to-End Validation: Baseline vs Bayesian Auto-Fix (NDCG/HitRate)</strong>
  </p>
</p>

---

This notebook rigorously tests GradTracer's active **Auto-Fix Mode**, which intervenes during training to scale gradients of oscillating (Zombie) embeddings based on Bayesian posteriors.

We move beyond simple MSE and prove that Auto-Fix leads to statistically significant improvements in standard RecSys **Ranking Metrics (HR@10, NDCG@10)** on the sparse MovieLens-100K dataset.

## 1. Setup & Data Loading

In [ ]:
!pip install torch pandas numpy scipy statsmodels gdown
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import urllib.request
import zipfile
from scipy import stats
from tqdm.auto import tqdm

# Download MovieLens-100K
url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
if not os.path.exists("ml-100k"):
    urllib.request.urlretrieve(url, "ml-100k.zip")
    with zipfile.ZipFile("ml-100k.zip", 'r') as zip_ref:
        zip_ref.extractall(".")

columns = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=columns)

# Implicit Feedback conversion (Rating >= 4 is positive)
df = df[df['rating'] >= 4].copy()
df['rating'] = 1.0

df['user_id'] = df['user_id'].astype('category').cat.codes
df['item_id'] = df['item_id'].astype('category').cat.codes
num_users = df['user_id'].nunique()
num_items = df['item_id'].max() + 1

# Leave-One-Out Evaluation Split
df = df.sort_values(by=['user_id', 'timestamp'])
test_df = df.groupby('user_id').tail(1)
train_df = df.drop(test_df.index)

class ImplicitMFDataset(Dataset):
    def __init__(self, df, num_items, num_negatives=4):
        self.users = torch.tensor(df['user_id'].values, dtype=torch.long)
        self.items = torch.tensor(df['item_id'].values, dtype=torch.long)
        self.labels = torch.ones(len(df), dtype=torch.float32)
        self.num_items = num_items
        self.num_negatives = num_negatives
        
    def __len__(self):
        return len(self.users)
        
    def __getitem__(self, idx):
        u = self.users[idx]
        i = self.items[idx]
        
        # Negative sampling
        neg_items = torch.randint(0, self.num_items, (self.num_negatives,))
        
        items_batch = torch.cat([i.unsqueeze(0), neg_items])
        labels_batch = torch.cat([torch.tensor([1.0]), torch.zeros(self.num_negatives)])
        users_batch = u.repeat(1 + self.num_negatives)
        
        return users_batch, items_batch, labels_batch

# Note: Collate function flattens the multi-negative batches
def flatten_collate(batch):
    users = torch.cat([item[0] for item in batch])
    items = torch.cat([item[1] for item in batch])
    labels = torch.cat([item[2] for item in batch])
    return users, items, labels

train_loader = DataLoader(
    ImplicitMFDataset(train_df, num_items), 
    batch_size=256, 
    shuffle=True, 
    collate_fn=flatten_collate
)

## 2. Model & Ranking Evaluator (HR@10, NDCG@10)

In [ ]:
class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, dim=32):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, dim)
        self.item_emb = nn.Embedding(num_items, dim)
        nn.init.normal_(self.user_emb.weight, std=0.01)
        nn.init.normal_(self.item_emb.weight, std=0.01)
        
    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        return (u * i).sum(dim=1)

def evaluate_ranking(model, test_df, train_df, num_items, k=10):
    model.eval()
    hits = []
    ndcgs = []
    
    # Build interaction dict for fast lookup (items to ignore)
    train_interacts = train_df.groupby('user_id')['item_id'].apply(set).to_dict()
    
    device = next(model.parameters()).device
    all_item_ids = torch.arange(num_items, device=device)
    
    with torch.no_grad():
        for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Evaluating"):
            u = int(row['user_id'])
            pos_item = int(row['item_id'])
            
            # Get all predictions for user
            u_tensor = torch.tensor([u] * num_items, device=device)
            scores = model(u_tensor, all_item_ids).cpu().numpy()
            
            # Mask items seen in training
            seen = train_interacts.get(u, set())
            scores[list(seen)] = -np.inf
            
            # Get top K items
            top_k_items = np.argsort(scores)[-k:][::-1]
            
            # Metrics
            if pos_item in top_k_items:
                hits.append(1)
                rank = np.where(top_k_items == pos_item)[0][0]
                ndcgs.append(1.0 / np.log2(rank + 2))
            else:
                hits.append(0)
                ndcgs.append(0)
                
    return hits, ndcgs

## 3. Training: Baseline vs Bayesian Auto-Fix
We train exactly the same Matrix Factorization architecture, but one wrapped in GradTracer's `FlowManager` with `auto_fix=True` that intercepts Zombie Embeddings dynamically.

In [ ]:
from gradtracer.analyzers.embedding import EmbeddingTracker
from gradtracer.analyzers.manager import FlowManager

LR = 0.05 # Purposefully high LR to cause sparse gradients to oscillate
EPOCHS = 3

def train_model(auto_fix=False):
    model = MatrixFactorization(num_users, num_items, dim=32)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.BCEWithLogitsLoss() # Implicit feedback
    
    manager = FlowManager()
    item_tracker = EmbeddingTracker(model.item_emb, name="item_emb", auto_fix=auto_fix, track_interval=20)
    manager.add_tracker("item", item_tracker)
    
    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        for users, items, labels in train_loader:
            optimizer.zero_grad()
            preds = model(users, items)
            loss = criterion(preds, labels)
            loss.backward()
            
            # AutoFix intervenes here during backwards pass via register_hook
            manager.step()
            
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")
        
    manager.report()
    return model

print("===============================")
print("🛑 Training BASELINE Model")
print("===============================")
model_baseline = train_model(auto_fix=False)

print("\n===============================")
print("✅ Training AUTO-FIX Model")
print("===============================")
model_autofix = train_model(auto_fix=True)

## 4. Statistical Validation (Rank Evaluation)
We now evaluate both models on HR@10 and NDCG@10, performing a paired t-test directly on the per-user NDCG scores to prove statistical significance.

In [ ]:
print("Evaluating Baseline...")
base_hits, base_ndcgs = evaluate_ranking(model_baseline, test_df, train_df, num_items)

print("Evaluating Auto-Fix...")
fix_hits, fix_ndcgs = evaluate_ranking(model_autofix, test_df, train_df, num_items)

base_hr, base_ndcg = np.mean(base_hits), np.mean(base_ndcgs)
fix_hr, fix_ndcg = np.mean(fix_hits), np.mean(fix_ndcgs)

# Paired T-test on NDCG Arrays
t_stat, p_val = stats.ttest_rel(base_ndcgs, fix_ndcgs)

print("\n=======================================================")
print("📊 Evaluation: Empirical Proof of Bayesian Auto-Fix")
print("=======================================================")
print(f"📉 Baseline       -> HR@10: {base_hr:.4f} | NDCG@10: {base_ndcg:.4f}")
print(f"📈 AUTO-FIX       -> HR@10: {fix_hr:.4f} | NDCG@10: {fix_ndcg:.4f}")
print(f"Improvement       -> HR: +{(fix_hr - base_hr) / base_hr * 100:.1f}%   | NDCG: +{(fix_ndcg - base_ndcg) / base_ndcg * 100:.1f}%")
print("-" * 55)
print(f"Paired t-test (NDCG) p-value: {p_val:.4e}")

if p_val < 0.05 and fix_ndcg > base_ndcg:
    print("✅ Conclusion: FlowGrad's Auto-Fix yields a STATISTICALLY SIGNIFICANT ranking improvement.")
else:
    print("❌ Conclusion: The Auto-Fix did not yield a statistically significant ranking improvement.")